## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob

## Read-in

In [2]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jun. 10th Jun. 17th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [6]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [7]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_30995/2994109523.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])


In [8]:
# Filter dates between. If only using data looking back one week, this shouldn't change the df
df = df[(df['SOLD DATE'] >= '2024-06-10') & (df['SOLD DATE'] <= '2024-06-17')]

In [10]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
96,PAST SALE,2024-06-17,Condo/Co-op,4854 NW 7th St #506,West Miami,FL,33126.0,335000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/4854-NW-7th-St...,MARMLS,A11529686,N,Y,25.778196,-80.275145
67,PAST SALE,2024-06-14,Condo/Co-op,1900 N Bayshore Dr #4502,Miami,FL,33132.0,815000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1900-N-Bayshor...,MARMLS,A11557357,N,Y,25.795043,-80.187129
39,PAST SALE,2024-06-14,Condo/Co-op,7000 Island Blvd #2605,Aventura,FL,33160.0,1065000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/7000-Island...,MARMLS,A11553848,N,Y,25.944168,-80.135482
28,PAST SALE,2024-06-14,Condo/Co-op,11309 SW 200th St Unit 103-C,Miami,FL,33157.0,195000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/11309-SW-200th...,MARMLS,A11570940,N,Y,25.582199,-80.374798
31,PAST SALE,2024-06-14,Condo/Co-op,3131 NE 7th Ave #1004,Miami,FL,33137.0,1400000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3131-NE-7th-Av...,MARMLS,A11568332,N,Y,25.807047,-80.185578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,PAST SALE,2024-06-10,Condo/Co-op,133 NE 2nd Ave #2409,Miami,FL,33132.0,365000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/133-NE-2nd-Ave...,MARMLS,A11532561,N,Y,25.775954,-80.190193
98,PAST SALE,2024-06-10,Condo/Co-op,2843 S Bayshore Dr Unit 17A,Coconut Grove,FL,33133.0,1000000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2843-S-Bayshor...,MARMLS,A11513177,N,Y,25.728269,-80.238889
22,PAST SALE,2024-06-10,Condo/Co-op,791 Crandon Blvd #407,Key Biscayne,FL,33149.0,4710000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,MARMLS,A11569824,N,Y,25.688337,-80.157932
101,PAST SALE,2024-06-10,Condo/Co-op,90 SW 3rd St #3406,Miami,FL,33130.0,560000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/90-SW-3rd-St-3...,MARMLS,A11499097,N,Y,25.770365,-80.195214


In [11]:
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

## Current Week's Values

In [12]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

133


In [13]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$852,258


In [14]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$569


In [15]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$113,350,362


In [54]:
# # Define the date range
# start_date = '2024-05-20'
# end_date = '2024-05-26'

# # Filter rows where 'SOLD DATE' is within the specified range
# second_week_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

## Previous Week's Values

In [16]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
189
Input Previous Week Condo Average Sales Price:
892000
Input Previous Week Condo Average PSF:
575
Input Previous Week Condo Sales total (ex: 198_000_000)
169000000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [17]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [18]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [19]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'RANK', 'COLOR'],
      dtype='object')

In [20]:
pd.set_option('display.max_columns',None)

In [21]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
78,PAST SALE,2024-06-14,Condo/Co-op,18975 Collins Ave #3700,Sunny Isles Beach,FL,33160.0,6400000.0,4.0,5.5,18975 COLLINS CONDO,3891.0,NaN,2019.0,NaN,1645.0,5550.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11542651,N,Y,25.95174,-80.119755,1,orange


In [22]:
def popup_html(row):
    building_name = row['LOCATION']
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [23]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [24]:
m.save('index.html')

## Data snagger

In [25]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
78,PAST SALE,2024-06-14,Condo/Co-op,18975 Collins Ave #3700,Sunny Isles Beach,FL,33160.0,6400000.0,4.0,5.5,18975 COLLINS CONDO,3891.0,NaN,2019.0,NaN,1645.0,5550.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11542651,N,Y,25.95174,-80.119755,1,orange


In [ ]:
# , {df_top_ten[df_top_ten['PRICE'].idxmax()]['ADDRESS']} | Price ${df_top_ten[df_top_ten['PRICE'].idxmax()]['PRICE']:,.0f}

In [27]:
df_top_ten['PRICE'].iloc[0]

6400000.0

In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [30]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_06172024


# CREATE TEMPLATE 

In [31]:
muni_set = set(df_top_ten['CITY'])

In [32]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [33]:
df_top_ten.reset_index(inplace=True,drop=True)

In [34]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [35]:
top_sale

'18975 COLLINS CONDO at 18975 Collins Ave #3700 in Sunny Isles Beach'

In [29]:
# previous_week_condo_sales_total = 130_000_000

In [36]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [111]:
# df_top_ten

## RESUME HERE

In [37]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} sales average from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 18975 COLLINS CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,400,000M to $6,400,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Bal Harbour, Sunny Isles Beach, Key Biscayne, Miami Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 133 condo sales totaling $113,350,362 million from Jun. 10th to Jun. 17th. The previous week, brokers closed 189 condo sales totaling $169,000,000.

Last week’s units sold for an average of $852,258, lower than the $892,000 sales average from the previous week. The avera

In [38]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [39]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [40]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf")

print(story_checklist)


HED: 18975 COLLINS CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,400,000M to $6,400,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Bal Harbour, Sunny Isles Beach, Key Biscayne, Miami Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 133 condo sales totaling $113,350,362 million from Jun. 10th to Jun. 17th. The previous week, brokers closed 189 condo sales totaling $169,000,000.

Last week’s units sold for an average of $852,258, lower than the $892,000 sales average from the previous week. The avera

In [49]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Sunny-Isles-Beach/18975-Collins-Ave-33160/unit-3700/home/171995967


In [50]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami-Beach/102-24th-St-33139/unit-1212/home/79136199


In [51]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami-Beach/1000-S-Pointe-Dr-33139/unit-1507/home/43276119


In [54]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Bal-Harbour/10201-Collins-Ave-33154/unit-802/home/56743737


In [55]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/6799-Collins-Ave-33141/unit-LPH2/home/190051321


In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-06-14,Condo/Co-op,18975 Collins Ave #3700,Sunny Isles Beach,FL,33160.0,6400000.0,4.0,5.5,18975 COLLINS CONDO,3891.0,NaN,2019.0,NaN,1645.0,5550.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11542651,N,Y,25.951740,-80.119755,1,orange,18975 COLLINS CONDO at 18975 Collins Ave #3700...
1,PAST SALE,2024-06-10,Condo/Co-op,10201 Collins Ave #802,Bal Harbour,FL,33154.0,6300000.0,2.0,3.0,OCEANA BAL HARBOUR CONDO,2121.0,NaN,2016.0,NaN,2970.0,3610.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11514106,N,Y,25.895168,-80.123150,2,blue,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
2,PAST SALE,2024-06-10,Condo/Co-op,10 Venetian Way Ph 1,Miami Beach,FL,33139.0,6000000.0,3.0,2.5,THE GRAND VENETIAN CONDO,4750.0,NaN,2001.0,NaN,1263.0,5320.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/10-Venet...,MARMLS,A11476722,N,Y,25.791132,-80.146640,3,blue,THE GRAND VENETIAN CONDO at 10 Venetian Way Ph...
3,PAST SALE,2024-06-12,Condo/Co-op,2234 Fisher Island Dr #3304,Miami Beach,FL,33109.0,5000000.0,3.0,3.5,BAYSIDE VILLAGE CONDO,3192.0,NaN,1986.0,NaN,1566.0,9299.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2234-Fis...,MARMLS,A11415796,N,Y,25.759225,-80.142843,4,blue,BAYSIDE VILLAGE CONDO at 2234 Fisher Island Dr...
4,PAST SALE,2024-06-10,Condo/Co-op,5959 Collins Ave #1107,Miami Beach,FL,33140.0,4855000.0,3.0,3.5,THE RESIDENCES AT THE BAT,2959.0,NaN,2005.0,NaN,1641.0,4558.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,MARMLS,A11486809,N,Y,25.841880,-80.120969,5,blue,THE RESIDENCES AT THE BAT at 5959 Collins Ave ...
5,PAST SALE,2024-06-10,Condo/Co-op,791 Crandon Blvd #407,Key Biscayne,FL,33149.0,4710000.0,4.0,5.5,OCEAN TOWER TWO CONDO,3581.0,NaN,2001.0,NaN,1315.0,4786.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,MARMLS,A11569824,N,Y,25.688337,-80.157932,6,blue,OCEAN TOWER TWO CONDO at 791 Crandon Blvd #407...
6,PAST SALE,2024-06-14,Condo/Co-op,6799 Collins Ave Unit LPH2,Miami Beach,FL,33141.0,3750000.0,4.0,4.0,SOUTH CARILLON BEACH COND,3041.0,NaN,2008.0,NaN,1233.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/6799-Collins-A...,MARMLS,A11551831,N,Y,25.852725,-80.120046,7,blue,SOUTH CARILLON BEACH COND at 6799 Collins Ave ...
7,PAST SALE,2024-06-11,Condo/Co-op,17121 Collins Ave #4308,Sunny Isles Beach,FL,33160.0,3200000.0,4.0,4.5,JADE OCEAN CONDO,2485.0,NaN,2009.0,NaN,1288.0,3668.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11448473,N,Y,25.934401,-80.121005,8,blue,JADE OCEAN CONDO at 17121 Collins Ave #4308 in...
8,PAST SALE,2024-06-11,Condo/Co-op,1000 S Pointe Dr #1507,Miami Beach,FL,33139.0,2500000.0,2.0,2.5,MURANO AT PORTOFINO CONDO,1390.0,NaN,2001.0,NaN,1799.0,1527.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11308118,N,Y,25.768527,-80.137591,9,blue,MURANO AT PORTOFINO CONDO at 1000 S Pointe Dr ...
9,PAST SALE,2024-06-13,Condo/Co-op,102 24th St #1212,Miami Beach,FL,33139.0,2400000.0,1.0,1.0,2399 COLLINS AVENUE,866.0,NaN,2015.0,NaN,2771.0,2325.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/102-24th...,MARMLS,A11572531,N,Y,25.799770,-80.126865,10,blue,2399 COLLINS AVENUE at 102 24th St #1212 in Mi...


## Time on Market Calculator

In [48]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 3, 1) ## List (Earlier) date
date2 = datetime(2024, 6, 14) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

105
